In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EcommerceWorkspace") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 17:30:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("ERROR")

In [4]:
SparkSession.builder.getOrCreate()

In [5]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS ecommerce
""")

DataFrame[]

In [6]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|ecommerce|
+---------+



In [8]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS ecommerce_data
""")

DataFrame[]

In [9]:
import os

os.environ["KAGGLE_USERNAME"] = "ernidhi"
os.environ["KAGGLE_KEY"] = "KGAT_1c8e2ae620ede4cb27a4f19a633c0a60"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [11]:
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
100%|██████████████████████████████████████▉| 4.28G/4.29G [00:04<00:00, 496MB/s]
100%|██████████████████████████████████████| 4.29G/4.29G [00:04<00:00, 1.03GB/s]


In [ ]:
!unzip ecommerce-behavior-data-from-multi-category-store.zip -d ecommerce_data

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: ecommerce_data/2019-Nov.csv  
  inflating: ecommerce_data/2019-Oct.csv  

In [13]:
!cd ecommerce_data && ls -lh

total 14G
-rw-r--r-- 1 root root 8.4G Dec  9  2019 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Dec  9  2019 2019-Oct.csv


In [14]:
!rm -f ecommerce-behavior-data-from-multi-category-store.zip

In [15]:
!ls -lh

total 16K
-rw-r--r-- 1 root root  684 Jan 11 17:31 derby.log
drwxr-xr-x 2 root root 4.0K Jan 11 17:36 ecommerce_data
drwxr-xr-x 5 root root 4.0K Jan 11 17:31 metastore_db
drwxr-xr-x 4 root root 4.0K Jan 11 17:32 spark-warehouse


In [16]:
!pwd
!ls


/kaggle/working
derby.log  ecommerce_data  metastore_db  spark-warehouse


In [17]:
!ls ecommerce_data

2019-Nov.csv  2019-Oct.csv


In [19]:
events = spark.read.csv(
    "ecommerce_data/*.csv",
    header=True,
    inferSchema=True
)

events.cache()
events.count()


109950743

In [20]:
## Left Join
products = events.select(
    "product_id", "category_code", "brand"
).dropDuplicates()

joined_df = events.join(
    products,
    on="product_id",
    how="left"
)

joined_df.show(10)


+----------+-------------------+----------+-------------------+--------------------+-------+-------+---------+--------------------+--------------------+-------+
|product_id|         event_time|event_type|        category_id|       category_code|  brand|  price|  user_id|        user_session|       category_code|  brand|
+----------+-------------------+----------+-------------------+--------------------+-------+-------+---------+--------------------+--------------------+-------+
|   1003285|2019-10-14 11:11:43|      view|2053013555631882655|electronics.smart...|samsung| 525.08|540201506|3ca13eed-acf1-49d...|electronics.smart...|samsung|
|   1003305|2019-10-31 17:04:02|      view|2053013555631882655|electronics.smart...|  apple| 599.23|566194367|5ef804c5-329f-4bd...|electronics.smart...|  apple|
|   1003310|2019-10-09 14:18:23|      view|2053013555631882655|electronics.smart...|  apple| 696.26|556265846|ec751875-9e6d-482...|electronics.smart...|  apple|
|   1003747|2019-10-29 09:17:33|  

In [21]:
## Running Total
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, col

window_spec = Window \
    .partitionBy("user_id") \
    .orderBy("event_time") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

events_with_running_sales = events.withColumn(
    "running_sales",
    sum(col("price")).over(window_spec)
)

events_with_running_sales.select(
    "user_id", "event_time", "price", "running_sales"
).show(10)


+---------+-------------------+------+------------------+
|  user_id|         event_time| price|     running_sales|
+---------+-------------------+------+------------------+
|116566414|2019-11-14 11:36:19| 205.9|             205.9|
|116566414|2019-11-24 15:10:27|   0.0|             205.9|
|116566414|2019-11-24 15:13:32|218.51|424.40999999999997|
|116566414|2019-11-24 15:14:39|   0.0|424.40999999999997|
|116566414|2019-11-24 15:16:52| 126.1|            550.51|
|116566414|2019-11-24 15:18:16|360.08| 910.5899999999999|
|116566414|2019-11-24 15:18:35| 126.1|1036.6899999999998|
|116566414|2019-11-24 15:18:40|   0.0|1036.6899999999998|
|116566414|2019-11-24 15:18:48|   0.0|1036.6899999999998|
|116566414|2019-11-24 15:19:40|185.31|1221.9999999999998|
+---------+-------------------+------+------------------+
only showing top 10 rows



In [23]:
## Ranking
from pyspark.sql.functions import dense_rank

rank_window = Window.partitionBy("category_code").orderBy(col("price").desc())

ranked_products = events.withColumn(
    "price_rank",
    dense_rank().over(rank_window)
)

ranked_products.select(
    "category_code", "product_id", "price", "price_rank"
).show(20)


+-------------+----------+------+----------+
|category_code|product_id| price|price_rank|
+-------------+----------+------+----------+
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400158|226.52|         1|
|apparel.jeans|  44400158|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400158|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400158|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.jeans|  44400157|226.52|         1|
|apparel.j

In [31]:
## User Defined Function
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def price_bucket(price):
    if price is None:
        return "Unknown"
    elif price <= 500:
        return "Low"
    elif price < 1000:
        return "Medium"
    else:
        return "High"

price_bucket_udf = udf(price_bucket, StringType())

events_udf = events.withColumn(
    "price_bucket",
    price_bucket_udf(col("price"))
)

events_udf.select("brand", "price", "price_bucket").show(20)


+--------+------+------------+
|   brand| price|price_bucket|
+--------+------+------------+
|  xiaomi|489.07|         Low|
|  janome|293.65|         Low|
|   creed| 28.31|         Low|
|      lg|712.87|      Medium|
|  xiaomi|183.27|         Low|
|      hp|360.09|         Low|
|      hp|514.56|      Medium|
| rondell| 30.86|         Low|
|michelin| 72.72|         Low|
|   apple|732.07|      Medium|
|    NULL|437.33|         Low|
|    NULL|155.11|         Low|
|    NULL| 31.64|         Low|
|    NULL| 20.54|         Low|
| samsung|411.83|         Low|
|    NULL|  1.09|         Low|
|    NULL|  8.55|         Low|
|  xiaomi| 22.68|         Low|
|  huawei|164.84|         Low|
|      hp|411.59|         Low|
+--------+------+------------+
only showing top 20 rows



In [33]:
## Created Derived Features
from pyspark.sql.functions import when

events_enriched = events.withColumn(
    "high_value_purchase",
    when(col("price") > 500, 1).otherwise(0)
)

events_enriched.select(
    "brand","price", "high_value_purchase"
).show(5)


+------+------+-------------------+
| brand| price|high_value_purchase|
+------+------+-------------------+
|xiaomi|489.07|                  0|
|janome|293.65|                  0|
| creed| 28.31|                  0|
|    lg|712.87|                  1|
|xiaomi|183.27|                  0|
+------+------+-------------------+
only showing top 5 rows

